# 周波数応答の数式処理

吉田勝俊（宇都宮大学）

## 参考情報

- [Pythonで数学の勉強：微分積分学 - Qiita](https://qiita.com/tibigame/items/aebbac176d9bbdaf3d15)
- [Jupyter Notebookでギリシャ文字を簡単に入力するには - Qiita](https://qiita.com/alchemist/items/0ce850770d8cc3df0ab4)

<!-- - [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)
- [[Python/matplotlib] FuncAnimationを理解して使う - Qiita](https://qiita.com/osanshouo/items/3c66781f41884694838b)
- [2つの信号間の遅延を推定する - Qiita](https://qiita.com/inoory/items/3ea2d447f6f1e8c40ffa) -->

In [ ]:
import sympy as sym #数式処理ライブラリ

In [ ]:
# 出力用のユーザ関数

from IPython.display import Markdown, display
def printmd(string):
    '''
    Markdown を出力する
    '''
    display(Markdown(string))

def display_all(var):
    '''
    ネストされた結果を，一列に表示する
    '''
    if len(var) == 0:
        return

    if type(var[0]) is list or type(var[0]) is tuple:
        display_all(var[0])
    else:
        display(var[0])
        
    display_all(var[1:])

## 正規形の運動方程式

In [ ]:
def EOM_can():
    '''
    強制振動系の正規形の運動方程式
    '''
    ### 数式処理用の文字変数 ###
    # 調和励振系のパラメータ
    om = sym.symbols(r'\omega', real=True)  #入力角振動数（この関数では未使用）
    zeta = sym.symbols(r'\zeta', real=True) #引数の文字列はdisplay表示用
    parameters = [zeta]

    # 時間関数
    t = sym.symbols(r'\tau', real=True)   #時間変数
    y = sym.Function(r'y', real=True)(t)  #変位 x(t)
    h = sym.Function(r'h', real=True)(t)  #外力 h(t)
    
    ### 運動方程式の定義 ... = 0 の左辺###
    # 1階微分 diff(x,t)，2階微分 diff(x,t,2)
    EOM = sym.diff(y,t,2) + 2*zeta*sym.diff(y,t) + y - h

    return (EOM, y, t, h, om, parameters)

display_all(EOM_can())

## 演習 6.2 ( ハーモニックバランス法の数式処理 )

### 三角関数によるハーモニックバランス

In [ ]:
def harmonic_balance_sin(EOM_func):
    '''
    sin によるハーモニックバランス
    '''
    # 運動方程式の読み込み
    EOM, y, t, h, om, params = EOM_func()

    printmd('**Equation of motion:**')
    display(EOM)

    # 調和入力
    A = sym.symbols(r'A')       #入力振幅
    ht = A*sym.sin(om*t)

    printmd('**Input:**')
    display(ht)

    # 定常応答
    B   = sym.symbols(r'B')     #出力振幅
    phi = sym.symbols(r'\phi')  #位相差
    yt  = B*sym.sin(om*t + phi) #定常応答

    printmd('**Output:**')
    display(yt)

    ### 運動方程式への代入 ###
    tmp = EOM
    tmp = tmp.replace(h, ht)    #入力の代入
    tmp = tmp.replace(y, yt)    #定常応答の代入
    tmp = tmp.simplify()        #一旦整理
    tmp = tmp.expand(trig=True) #三角関数の展開

    ### 連立方程式を解く ###
    # 単振動の係数
    sin_om, cos_om = sym.sin(om*t), sym.cos(om*t)
    coeff_sin_om = sym.collect(tmp, sin_om).coeff(sin_om)
    coeff_cos_om = sym.collect(tmp, cos_om).coeff(cos_om)
    
    printmd('**Multipliers of simple oscillations:**')
    display(coeff_sin_om, coeff_cos_om)
    
    # 連立方程式を解く
    a_real, b_real = B*sym.cos(phi), B*sym.sin(phi)
    
    sol = sym.solve(
        [sym.Eq(coeff_sin_om, 0), sym.Eq(coeff_cos_om, 0)],
        [a_real, b_real]
    )
    
    a, b = sol[a_real], sol[b_real] #連立方程式の解

    printmd('**Solution of harmonic balance:**')
    display(a,b)

    ### 周波数応答 ###
    tmp = (a**2 + b**2)/(A**2)
    tmp = tmp.simplify()
    barR = sym.sqrt(tmp)
    
    tmp = b/a
    tmp = tmp.simplify()
    phi = sym.atan(tmp)

    printmd('**Frequency response:**')
    display(barR, phi)
    
    return (barR, phi, om, params)

harmonic_balance_sin(EOM_can); # ; で返り値の出力を抑制

### 極形式によるハーモニックバランス

In [ ]:
def my_arg(comlex_number):
    '''
    複素数の偏角を返す自作関数 ※組込の sympy.arg の代替
    （sympy.arg は simplify が効かないかも．実体が atan2 だから？）
    '''
    real_part = sym.re(comlex_number)
    imag_part = sym.im(comlex_number)
    
    slope = (imag_part/real_part).simplify()
    angle = sym.atan(slope)
    
    return angle

In [ ]:
def harmonic_balance_complex(EOM_func):
    '''
    極形式によるハーモニックバランス
    '''
    # 運動方程式の読み込み
    EOM, y, t, h, om, params = EOM_func()

    printmd('**Equation of motion:**')
    display(EOM)

    ### 入出力の定義 ###
    jj = sym.I                       #虚数単位

    # 調和入力
    A = sym.symbols(r'A', real=True) #入力振幅
    ht = A*sym.exp( (om*t)*jj )      #調和入力

    printmd('**Input:**')
    display(ht)

    # 定常応答
    B   = sym.symbols(r'B', real=True)    #出力振幅
    phi = sym.symbols(r'\phi', real=True) #位相差
    yt  = B*sym.exp( (om*t + phi)*jj )    #定常応答
    
    printmd('**Output:**')
    display(yt)

    ### 運動方程式への代入 ###
    tmp = EOM
    tmp = tmp.replace(h, ht)        #入力の代入
    tmp = tmp.replace(y, yt)        #定常応答の代入
    tmp = tmp/ht                    #入力で割る
    tmp = tmp.simplify().expand()   #整理
    EOM_om = tmp
    
    printmd('**Equation of motion in frequency domain:**')
    display(EOM_om)
    
    ### 複素振幅比を求める ###
    R   = sym.symbols(r'\bar{R}', real=True) #振幅比の変数記号
    eqn = EOM_om.replace(B, R*A).simplify()  #代入
    
    #複素振幅比
    R_e_phi_j = sym.symbols(r'R*exp(phi*jj)')              #solve用の未知数
    eqn       = eqn.replace(sym.exp(phi*jj), R_e_phi_j/R)  #未知数による方程式の表示
    sol       = sym.solve(eqn, R_e_phi_j, force=True)      #求解
    R_e_phi_j = sol[0] #解が1要素のリストで返ってきたので，その中身

    printmd('**Complex amplitude ratio:**')
    display(R_e_phi_j)
    
    ### 周波数応答 ###
    barR = sym.Abs(R_e_phi_j)
#    phi = sym.arg(R_e_phi_j) #内部でatan2 を使ってるようで計算結果が整理されない
    phi_atan  = my_arg(R_e_phi_j) #人間用に簡約したバージョン

    printmd('**Frequency response:**')
    display(barR, phi_atan)
    
    return (R_e_phi_j, barR, phi, om, params)

harmonic_balance_complex(EOM_can); # ; で返り値の出力を抑制

## 演習 6.3 ( ラプラス変換の数式処理 )

- ラプラス変換の定義式：$P(s):=\int_{0}^{\infty}p(t)\exp(-s t)dt$

In [ ]:
def Laplace_trans(ft, t, s):
    '''
    ラプラス変換する関数
    '''
    Ps = sym.integrate(ft*sym.exp(-s*t), (t, 0, sym.oo))
    Ps = Ps.simplify() #式の整理
    
    return Ps

In [ ]:
def display_Laplace_trans():
    '''
    ラプラス変換表を導出する関数
    '''
    #数式処理用の文字変数
    t, s, ω, ϕ = sym.symbols('t s \omega \phi', Real=True, positive=True)

    #数式処理用の関数名
    x = sym.Function('x')
    
    #変換対象の時間関数
    time_functions = [
        sym.sin(ω*t),
        sym.cos(ω*t),        
        sym.sin(ω*t + ϕ),
        sym.cos(ω*t + ϕ),
        sym.diff(x(t),t),
    ]
    
    for pt in time_functions:
        Ps = Laplace_trans(pt, t, s)
        printmd('---')
        display(pt)
        display(Ps)


display_Laplace_trans()

- 時間微分のは計算してくれませんでした．（できる方法もあるのかな？）

## 演習 6.6 ( 数式処理結果のボード線図 )

### 周波数応答の数式処理結果（再掲）

In [ ]:
Gjom, R, phi, om, params = harmonic_balance_complex(EOM_can)
display_all(params)

### 複素振幅比をNumpy関数に変換

In [ ]:
from sympy.utilities.lambdify import lambdify #数式をNumpy関数に変換するライブラリ関数
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
display(Gjom)
numpy_Gjom  = lambdify((om, *params), Gjom, 'numpy')

### 周波数応答の計算とプロット

In [ ]:
oms = np.linspace(0.5, 2.0, 200)
zeta = 0.1

# 周波数応答の計算
Gjoms = numpy_Gjom( oms, zeta ) # Gjom(om) := G(j*om)
Rs   = np.abs(Gjoms)            #振幅比のデータ列
phis = np.angle(Gjoms)          #位相差のデータ列...Numpy の arg は angle

# プロット
fig, ax = plt.subplots(1,2, figsize=(8,4))
ax[0].plot(oms, Rs)
ax[1].plot(oms, phis)
ax[0].set_ylabel(r'$\bar R$', fontsize=16)
ax[1].set_ylabel(r'$\phi$',   fontsize=16)
for a in ax:
    a.set_xlabel(r'$\omega$', fontsize=16)

fig.tight_layout()

## 演習 6.7 ( 伝達関数から求めた周波数応答 )

### 以上と同じ調和励振系の伝達関数（最初からNumpyで記述）

In [ ]:
# 伝達関数の定義
def _G(s, zeta):
    '''
    正規化された調和励振系の伝達関数
    '''
    return 1/(s**2 + 2*zeta*s + 1)

# Numpy 特有のプログラミング技法
G = np.vectorize(_G, excluded=['zeta']) #ベクトル引数を受付ける関数に拡張．zeta部分は除く

# 周波数応答
Rs2 = np.abs(G( (1j)*oms, zeta ))     #振幅比 |G(j*om)|
phis2 = np.angle(G( (1j)*oms, zeta )) #位相差 arg[G(j*om)]...Numpy の arg は angle

# プロット
fig, ax = plt.subplots(1,2, figsize=(8,4))
ax[0].plot(oms, Rs2)
ax[1].plot(oms, phis2)
ax[0].set_ylabel(r'$\bar R$', fontsize=16)
ax[1].set_ylabel(r'$\phi$',   fontsize=16)
for a in ax:
    a.set_xlabel(r'$\omega$', fontsize=16)

fig.tight_layout()

- ほんの数行で，周波数応答が求まります．